In [2]:
import pandas as pd
import pyodbc
import numpy as np
import os

In [22]:
path = r'T:\Tableau\tableauJourneyToWork\Datasources'

In [81]:
# adjust accordingly on the tables ModeShare_ALL_Years; ModeByVehiclesAvailable_AllYears; 
# ModeByPovertyStatus_AllYears; TimeLeavingForWork_AllYears; TravelTimeToWork_AllYears
data = pd.read_csv(path + '\\TimeLeavingForWork_AllYears.csv')   
df = pd.DataFrame(data)

print(df)

     Year             Geography Time.Leaving.for.Work..Census.  \
0    2000                Coburg        12:00 a.m. to 4:59 a.m.   
1    2000                Coburg         5:00 a.m. to 5:29 a.m.   
2    2000                Coburg         5:30 a.m. to 5:59 a.m.   
3    2000                Coburg         6:00 a.m. to 6:29 a.m.   
4    2000                Coburg         6:30 a.m. to 6:59 a.m.   
..    ...                   ...                            ...   
975  2021  Salem Urbanized Area         9:00 a.m. to 9:59 a.m.   
976  2021  Salem Urbanized Area       10:00 a.m. to 10:59 a.m.   
977  2021  Salem Urbanized Area       11:00 a.m. to 11:59 a.m.   
978  2021  Salem Urbanized Area        12:00 p.m. to 3:59 p.m.   
979  2021  Salem Urbanized Area        4:00 p.m. to 11:59 p.m.   

    Time.Leaving.for.Work  Estimate  MOE_Est     Share  MOE_Share   SharePCT  \
0        Midnight to 5 AM        18      NaN       NaN        NaN   4.090909   
1            5 to 5:30 AM        14      NaN   

In [82]:
df.dtypes

Year                                int64
Geography                          object
Time.Leaving.for.Work..Census.     object
Time.Leaving.for.Work              object
Estimate                            int64
MOE_Est                           float64
Share                             float64
MOE_Share                         float64
SharePCT                          float64
MOE_SharePct                      float64
dtype: object

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [66]:
df['Length.of.Commute'].unique()

array(['Less than 5 minutes', '5 to 9 minutes', '10 to 14 minutes',
       '15 to 19 minutes', '20 to 24 minutes', '25 to 29 minutes',
       '30 to 34 minutes', '35 to 39 minutes', '40 to 44 minutes',
       '45 to 59 minutes', '60 to 89 minutes', '90 or more minutes'],
      dtype=object)

In [67]:
len('90 or more minutes')

18

In [83]:
df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)

In [84]:
df = df.fillna(0)

In [85]:
# table names: mode_share, mode_by_vehicles_available, mode_by_poverty_status, time_leaving_for_work, travel_time_to_work
# rewrite tables: DROP TABLE mode_share;
# edit accordingly the table
cursor.execute('''
            CREATE TABLE time_leaving_for_work_test(
            year int,
            geo varchar(50),
            time_leaving varchar(50),
            formatted_time varchar(50),
            estimate int,
            moe_est int,
            share float,
            moe_share float,
            share_pct float,
            moe_share_pct float
            )
               ''')

In [86]:
#df.rename(columns={'Mode.by.Vehicle.Availability': 'Mode', 'Vehicles.Available': 'VHCL_AVAIL'}, inplace=True)
#df.rename(columns={'Travel.Mode': 'Mode', 'Poverty.Status': 'Poverty_Status'}, inplace=True)
df.rename(columns={'Time.Leaving.for.Work..Census.': 'Time_Leaving', 'Time.Leaving.for.Work': 'FormattedTime'}, inplace=True)
#df.rename(columns={'Length.of.Commute': 'length_of_commute'}, inplace=True)

In [87]:
ndf = df[df.Year != 2021]

In [90]:
# mode_share, mode_by_vehicles_available, mode_by_poverty_status, time_leaving_for_work, travel_time_to_work
for row in ndf.itertuples():
    cursor.execute('''
                INSERT INTO time_leaving_for_work_test(year, geo, time_leaving, formatted_time, estimate, moe_est, share, moe_share, share_pct, moe_share_pct)
                VALUES (?,?,?,?,?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Geography,
                   row.Time_Leaving,
                   row.FormattedTime,
                   row.Estimate,
                   row.MOE_Est,
                   row.Share,
                   row.MOE_Share,
                   row.SharePCT,
                   row.MOE_SharePct
                  )
conn.commit()